In [ ]:
import cv2
import time
import math as m
import mediapipe as mp

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    """Calculate the Euclidean distance between two points."""
    return m.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def findAngle(x1, y1, x2, y2):
    """Calculate the angle between two points."""
    if y1 == y2:
        return 0

    delta_x = x2 - x1
    delta_y = y2 - y1
    theta = m.atan2(delta_y, delta_x)
    degree = abs(theta * (180 / m.pi)) 
    return degree

def sendWarning():
    """Placeholder function for sending an alert."""
    print("Warning: Bad posture detected for too long!")

file_name = 'NeckPosture.avi'
cap = cv2.VideoCapture(file_name)

if not cap.isOpened():
    print("Error: Couldn't open video file.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'MJPG')

video_output = cv2.VideoWriter('output.avi', fourcc, fps, frame_size)

font = cv2.FONT_HERSHEY_SIMPLEX
green = (127, 255, 0)
red = (50, 50, 255)
yellow = (0, 255, 255)
pink = (255, 0, 255)

good_frames = 0
bad_frames = 0

pose = mp_pose.Pose()

print("Processing...")

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("End of video.")
        break

    h, w = image.shape[:2]

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)

    image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # Detect landmarks
    if keypoints.pose_landmarks:
        lm = keypoints.pose_landmarks.landmark
        lmPose = mp_pose.PoseLandmark

        l_shldr_x, l_shldr_y = int(lm[lmPose.LEFT_SHOULDER].x * w), int(lm[lmPose.LEFT_SHOULDER].y * h)
        r_shldr_x, r_shldr_y = int(lm[lmPose.RIGHT_SHOULDER].x * w), int(lm[lmPose.RIGHT_SHOULDER].y * h)
        l_ear_x, l_ear_y = int(lm[lmPose.LEFT_EAR].x * w), int(lm[lmPose.LEFT_EAR].y * h)
        l_hip_x, l_hip_y = int(lm[lmPose.LEFT_HIP].x * w), int(lm[lmPose.LEFT_HIP].y * h)

        # Calculate shoulder offset
        offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        alignment_status = "Aligned" if offset < 100 else "Not Aligned"
        #cv2.putText(image, f"{int(offset)} {alignment_status}", (w - 200, 30), font, 0.9, green if offset < 100 else red, 2)

        neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
        torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

        cv2.circle(image, (l_shldr_x, l_shldr_y), 7, yellow, -1)
        cv2.circle(image, (l_ear_x, l_ear_y), 7, yellow, -1)
        cv2.circle(image, (l_hip_x, l_hip_y), 7, yellow, -1)

        angle_text = f"Neck: {int(neck_inclination)} degrees"
        if neck_inclination < 40 and torso_inclination < 10:
            bad_frames = 0
            good_frames += 1
            color = green
        else:
            good_frames = 0
            bad_frames += 1
            color = red

        cv2.putText(image, angle_text, (10, 30), font, 0.9, color, 2)
        cv2.putText(image, str(int(neck_inclination)), (l_shldr_x + 10, l_shldr_y), font, 0.9, color, 2)
        cv2.putText(image, str(int(torso_inclination)), (l_hip_x + 10, l_hip_y), font, 0.9, color, 2)

        cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), color, 4)
        cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), color, 4)

        good_time = (1 / fps) * good_frames
        bad_time = (1 / fps) * bad_frames

        # Send warning if bad posture is held for too long
        if bad_time > 180:
            sendWarning()

    video_output.write(image)

print("Finished processing.")
cap.release()
video_output.release()
cv2.destroyAllWindows()


Processing...


I0000 00:00:1739852993.404438 4131831 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
W0000 00:00:1739852993.471320 4142321 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1739852993.486105 4142321 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


End of video.
Finished processing.
